This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [41]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [5]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/metadata.json?api_key={}".format(API_KEY))

In [6]:
print(r.content)

b'{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.\\u003cbr\\u003e\\u003cbr\\u003eTrading System: Xetra\\u003cbr\\u003e\\u003cbr\\u003eISIN: DE0005313704","refreshed_at":"2020-12-01T14:48:09.907Z","newest_available_date":"2020-12-01","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"database_id":6129}}'


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
# (keep in mind that the date format is YYYY-MM-DD).

r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={}".format(API_KEY))

In [15]:
# 2. Convert the returned JSON object into a Python dictionary.

import json
data = json.loads(r.content)

In [25]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

# list comprehension to collect all high and low prices into separate lists
high_prices = [x[2] for x in data["dataset_data"]["data"]]
low_prices = [x[3] for x in data["dataset_data"]["data"]]

# print results for max and min
print("Highest price in 2017 is: " + str(max(high_prices)))
print("Lowest price in 2017 is: " + str(min(low_prices)))

Highest price in 2017 is: 53.54
Lowest price in 2017 is: 33.62


In [29]:
# 4. What was the largest change in any one day (based on High and Low price)?

# list comprehension to collect all differences into a new list
differences = [a-b for a,b in zip(high_prices,low_prices)]

# print max difference (rounded)
print("Biggest price change in a single day in 2017 is: " + str(round(max(differences),2)))

Biggest price change in a single day in 2017 is: 2.81


In [31]:
# 5. What was the largest change between any two days (based on Closing Price)?

# list comprehension to collect all closing prices into a list
close_prices = [x[4] for x in data["dataset_data"]["data"]]

# list comprehension to collect all differences into a new list
close_diff = [x-y for x,y in zip(close_prices[:-1],close_prices[1:])]

# print max difference (rounded) between days
print("Largest change between two days in 2017 is: " + str(round(max(close_diff),2)))

Largest change between two days in 2017 is: 1.72


In [35]:
# 6. What was the average daily trading volume during this year?

# list comprehension to collect all trading volumes
volume = [x[6] for x in data["dataset_data"]["data"]]

# print average volume (rounded)
print("Average daily trading volume in 2017 is: " + str(round(sum(volume)/len(volume),2)))

Average daily trading volume in 2017 is: 89124.34


In [42]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement 
# your own function for calculating the median.)

# sorts all volumes
volume.sort()

# Because there is an odd number of days we just take the middle value of the sorted list
# print median volume
print("Median daily trading volume in 2017 is: " + str(volume[round((len(volume)/2))]))

Median daily trading volume in 2017 is: 76600.0
